In [2]:
# !pip install faiss-cpu boto3

  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


In [4]:
import boto3
import faiss
import json
import numpy as np

# Load Vector db

In [5]:
s3_client = boto3.client('s3')

In [6]:
bucket_name = 'sageon-mungchi-service'
key = 'vector_db/'

In [7]:
def get_s3_file_keys():
    list_response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=key)
    file_keys = [content['Key'] for content in list_response.get('Contents', [])]
    return file_keys

In [8]:
def get_latest_index_and_matadata_key(keys):
    key_dts = [int(key_string.split('/')[2]) for key_string in keys]
    latest_dt = max(key_dts)

    latest_index_key = f'{key}faiss/{latest_dt}/faiss_index.index'
    latest_metadata_key = f'{key}faiss/{latest_dt}/metadata.json'

    return latest_index_key, latest_metadata_key

In [9]:
def get_index(index_key):
    local_filename = 'faiss_index.index'
    s3_client.download_file(bucket_name, index_key, local_filename)
    return faiss.read_index(local_filename)

In [10]:
def get_metadata(metadata_key):
    local_filename = 'metadata.json'
    s3_client.download_file(bucket_name, metadata_key, local_filename)

    with open(local_filename, 'r') as f:
        metadata = json.load(f)

    return metadata

In [11]:
def get_vectordb_and_metadata():
    keys = get_s3_file_keys()
    index_key, metadata_key = get_latest_index_and_matadata_key(keys)
    index = get_index(index_key)
    metadata = get_metadata(metadata_key)
    return index, metadata

In [12]:
index, metadata = get_vectordb_and_metadata()

In [13]:
metadata['3']

{'case_no': '2012두6551',
 'case_name': '취득세등부과처분취소',
 'case_date': '2012.07.12',
 'case_court': '대법원',
 'text': '과밀억제권역 안에서 본점 또는 주사무소용 건축물을 신축 또는 증축하여 취득하는 경우, 동일한 과밀억제권역 안에 있던 기존의 본점 또는 주사무소에서 이전해 오더라도 구 지방세법 제112조 제3항에 의한 취득세 중과대상인지 여부(적극) '}

# 벡터 데이터베이스 검색 결과

In [14]:
br_client = boto3.client("bedrock-runtime", region_name="ap-northeast-1")

In [15]:
def get_cohere_embedding(text):
    model_id = "cohere.embed-multilingual-v3"
    native_request = {"texts": [text[0:2048]], 'input_type':'search_query'}
    request = json.dumps(native_request)
    response = br_client.invoke_model(modelId=model_id, body=request)
    model_response = json.loads(response["body"].read())
    embedding = model_response['embeddings'][0]
    return embedding

In [16]:
def get_vectordb_query_result(query, top_n=5):
    query_embedding = get_cohere_embedding(query)
    distances, indices = index.search(np.array([query_embedding]), top_n)

    result = []
    for i, idx in enumerate(indices[0]):
        result.append(metadata[str(idx)])

    return result

In [17]:
query = '사무실을 같은 동네에서 이전했는데 새로운 건물에 가격이 더 비싸다고 더 세금을 많이 내야 해?'

In [18]:
search_result = get_vectordb_query_result(query)
search_result[0]

{'case_no': '2012두6551',
 'case_name': '취득세등부과처분취소',
 'case_date': '2012.07.12',
 'case_court': '대법원',
 'text': '과밀억제권역 안에서 본점 또는 주사무소용 건축물을 신축 또는 증축하여 취득하는 경우, 동일한 과밀억제권역 안에 있던 기존의 본점 또는 주사무소에서 이전해 오더라도 구 지방세법 제112조 제3항에 의한 취득세 중과대상인지 여부(적극) '}

# 검색 결과로 생성형 AI에서 답변 생성하기

In [100]:
def generate_prompt(query, search_result, conversation_history=[]):
    context = ''
    for r in search_result:
        context += f"[사건번호: {r['case_no']}, 사건명: {r['case_name']}, 선고일자: {r['case_date']}, 법원종류: {r['case_court']}]\n"
        context += f"{r['text']}\n\n"

    history = ''
    for h in conversation_history:
        history += f"Human: {h['human']}\nAssistant: {h['assistant']}\n"
    
    prompt = f"""
    Human: 너는 한국 세법 전문가로 판례를 해석해서 사용자에게 필요한 세무 정보를 제공하는 AI assistant야.

    아래 주어지는 정보들은 이전 대화 내용이야.
    {history}
    
    아래 주어지는 정보들은 사용자의 질문과 관련된 판례 정보야.
    {context}

    이 판례 정보를 활용해 사용자 질문에 답변하되 없는 정보를 만들어내지 마.
    답변을 생성할 때 참고한 정보의 출처를 반드시 [사건번호: <참고 내용의 사건번호>, 사건명: <참고 내용의 사건명>, 선고일자: <참고 내용의 선고일자>, 법원종류: <참고 내용의 법원>] 형식으로 표기해.
    
    Human: {query}
    Assistant:
    """
    return prompt

In [89]:
def generate_answer(prompt):
    model_id = 'anthropic.claude-v2:1'

    body = json.dumps({
        'anthropic_version': 'bedrock-2023-05-31',
        'max_tokens': 10000,
        'messages': [
            {
                'role': 'user',
                'content': [{'type': 'text', 'text': prompt}]
            }
        ],
        'temperature': 0.5,
    })

    response = br_client.invoke_model(body=body, modelId=model_id, 
                                accept='application/json', contentType='application/json')

    response_body = json.loads(response.get('body').read())
    print(response_body)
    return response_body['content'][0]['text']

In [90]:
def generate_stream_answer(prompt):
    model_id = 'anthropic.claude-v2:1'

    body = json.dumps({
        'anthropic_version': 'bedrock-2023-05-31',
        'max_tokens': 10000,
        'messages': [
            {
                'role': 'user',
                'content': [{'type': 'text', 'text': prompt}]
            }
        ],
        'temperature': 0.5,
    })

    response = br_client.invoke_model_with_response_stream(
        body=body, modelId=model_id,
    )

    stream = response.get('body')
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_data = json.loads(chunk.get('bytes').decode())
                if chunk_data['type'] == 'content_block_delta':
                    yield chunk_data['delta']['text']

In [91]:
query1 = '사무실을 같은 동네에서 이전했는데 새로운 건물에 가격이 더 비싸다고 더 세금을 많이 내야 해?'
query2 = '그럼 어떤 법령을 기준으로 돈을 내야 하는건지 자세히 설명해봐'

In [102]:
conversation_history = []

In [103]:
# 1
print('Prompt:')
prompt = generate_prompt(query1, search_result, conversation_history)
print(prompt)
print()

print('Answer:')
answer = ''
for chunk in generate_stream_answer(prompt):
    print(chunk, end='')
    answer += chunk
print()

print('History:')
conversation_history.append({'human': query1, 'assistant': answer})
print(conversation_history)
print()

Prompt:

    Human: 너는 한국 세법 전문가로 판례를 해석해서 사용자에게 필요한 세무 정보를 제공하는 AI assistant야.

    아래 주어지는 정보들은 이전 대화 내용이야.
    
    
    아래 주어지는 정보들은 사용자의 질문과 관련된 판례 정보야.
    [사건번호: 2012두6551, 사건명: 취득세등부과처분취소, 선고일자: 2012.07.12, 법원종류: 대법원]
과밀억제권역 안에서 본점 또는 주사무소용 건축물을 신축 또는 증축하여 취득하는 경우, 동일한 과밀억제권역 안에 있던 기존의 본점 또는 주사무소에서 이전해 오더라도 구 지방세법 제112조 제3항에 의한 취득세 중과대상인지 여부(적극) 

[사건번호: 2013두26194, 사건명: 법인세할주민세처분취소, 선고일자: 2014.04.10, 법원종류: 대법원]
 법인의 사업장이 2 이상의 시·군에 소재하는 경우, 법인이 각 시·군에 납부할 세액을 정한 구 지방세법 시행령 제130조의5 제1항의 산식 중 ‘당해 시·군 내 건축물 연면적’과 ‘법인의 총건축물 연면적’이 동일한 기준에 의하여 산정되어야 하는지 여부(적극) 및 어느 시·군의 법인세할 주민세 부과처분이 확정되어 다툴 수 없게 된 상황에서 다른 시·군에 납부하여야 할 법인세할 주민세를 계산할 경우, 확정된 시·군에 소재하는 건축물의 연면적을 확정된 부과처분에서 적용한 기준이 아니라 법령에서 정한 정당한 기준에 따라 산정하여 ‘법인의 총건축물 연면적’을 계산하여야 하는지 여부(적극)

[사건번호: 2012두6551, 사건명: 취득세등부과처분취소, 선고일자: 2012.07.12, 법원종류: 대법원]
구 지방세법(2010. 3. 31. 법률 제10221호로 전부 개정되기 전의 것, 이하 같다) 제112조 제3항은 1998. 12. 31. 법률 제5615호로 개정되기 전과 달리 입법 취지를 반영하여 과밀억제권역 안에서 본점 또는 주사무소의 사업용 부동산을 취득하는 경우 중 인구유입과 산업집중의 효과가 뚜렷한 신축 또는 증축

In [104]:
# 2
print('Prompt:')
prompt = generate_prompt(query2, search_result, conversation_history)
print(prompt)
print()

print('Answer:')
answer = ''
for chunk in generate_stream_answer(prompt):
    print(chunk, end='')
    answer += chunk
print()

print('History:')
conversation_history.append({'Human': query1, 'Assistant': answer})
print(conversation_history)
print()

Prompt:

    Human: 너는 한국 세법 전문가로 판례를 해석해서 사용자에게 필요한 세무 정보를 제공하는 AI assistant야.

    아래 주어지는 정보들은 이전 대화 내용이야.
    Human: 사무실을 같은 동네에서 이전했는데 새로운 건물에 가격이 더 비싸다고 더 세금을 많이 내야 해?
Assistant: <사건번호: 2012두6551, 사건명: 취득세등부과처분취소, 선고일자: 2012.07.12, 법원종류: 대법원>

과밀억제권역 안에서 본점 또는 주사무소용 건축물을 신축 또는 증축하여 취득하는 경우, 동일한 과밀억제권역 안에 있던 기존의 본점 또는 주사무소에서 이전해 오더라도 구 지방세법 제112조 제3항에 의한 취득세 중과대상에 해당합니다. 

따라서 같은 동네 내에서 사무실 건물을 신축 또는 증축하여 취득할 경우, 기존 사무실에서 이전하더라도 취득세 중과세 대상이 될 수 있습니다.

    
    아래 주어지는 정보들은 사용자의 질문과 관련된 판례 정보야.
    [사건번호: 2012두6551, 사건명: 취득세등부과처분취소, 선고일자: 2012.07.12, 법원종류: 대법원]
과밀억제권역 안에서 본점 또는 주사무소용 건축물을 신축 또는 증축하여 취득하는 경우, 동일한 과밀억제권역 안에 있던 기존의 본점 또는 주사무소에서 이전해 오더라도 구 지방세법 제112조 제3항에 의한 취득세 중과대상인지 여부(적극) 

[사건번호: 2013두26194, 사건명: 법인세할주민세처분취소, 선고일자: 2014.04.10, 법원종류: 대법원]
 법인의 사업장이 2 이상의 시·군에 소재하는 경우, 법인이 각 시·군에 납부할 세액을 정한 구 지방세법 시행령 제130조의5 제1항의 산식 중 ‘당해 시·군 내 건축물 연면적’과 ‘법인의 총건축물 연면적’이 동일한 기준에 의하여 산정되어야 하는지 여부(적극) 및 어느 시·군의 법인세할 주민세 부과처분이 확정되어 다툴 수 없게 된 상황에서 다른 시·군에 납부하여야 할 법인세할 주민세를 계산할 경우, 확정된 시